# QSVM for AdHoc Dataset
## Prepare data

In [10]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from qiskit import Aer
from qiskit.algorithms.state_fidelities import ComputeUncompute
from qiskit.primitives import Sampler
from qiskit.providers.ibmq import least_busy
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit.circuit.library import ZZFeatureMap, PauliFeatureMap
from qiskit_machine_learning.kernels import QuantumKernel, FidelityQuantumKernel
from sklearn.model_selection import RandomizedSearchCV, cross_validate
from sklearn.svm import SVC

In [11]:
df = pd.read_csv('datasets/custom_dataset_1k.csv', index_col=0)
df

,feature_1,feature_2,label
0,2.619963,3.380037,0.0
1,3.363802,2.636198,0.0
2,3.326210,3.326210,0.0
3,3.351225,2.648775,0.0
4,2.552534,3.447466,0.0
...,...,...,...
995,2.643016,3.014067,1.0
996,2.686536,3.231210,1.0
997,3.222644,3.050097,1.0
998,3.291154,2.612386,1.0


In [12]:
feature_dimension = df.shape[1] - 1
print(f"Feature dimension: {feature_dimension}")

Feature dimension: 2


In [13]:
# Split the data into training and test sets
all_features = df.copy()
all_labels = all_features.pop('label')

train = df.sample(frac=0.75, random_state=42)
test = df.drop(train.index)

# Separate the features from the labels
train_features = train.copy()
test_features = test.copy()

train_labels = train_features.pop('label')
test_labels = test_features.pop('label')

## Find best SVM setting

Try different kernels with different settings and plot the winning circuit

In [14]:
algorithm_globals.random_seed = 42
quantum_instance = QuantumInstance(Aer.get_backend("aer_simulator"), shots=1024)

In [15]:
# Specify kernel experiments:

kernels = [
    QuantumKernel(feature_map=(ZZFeatureMap(feature_dimension=feature_dimension, reps=1, entanglement='linear')),
                  quantum_instance=quantum_instance),
    QuantumKernel(feature_map=(ZZFeatureMap(feature_dimension=feature_dimension, reps=2, entanglement='linear')),
                  quantum_instance=quantum_instance),
    QuantumKernel(feature_map=(ZZFeatureMap(feature_dimension=feature_dimension, reps=3, entanglement='linear')),
                  quantum_instance=quantum_instance),
    QuantumKernel(feature_map=(ZZFeatureMap(feature_dimension=feature_dimension, reps=4, entanglement='linear')),
                  quantum_instance=quantum_instance),
    QuantumKernel(feature_map=(ZZFeatureMap(feature_dimension=feature_dimension, reps=1, entanglement='sca')),
                  quantum_instance=quantum_instance),
    QuantumKernel(feature_map=(ZZFeatureMap(feature_dimension=feature_dimension, reps=2, entanglement='sca')),
                  quantum_instance=quantum_instance),
    QuantumKernel(feature_map=(ZZFeatureMap(feature_dimension=feature_dimension, reps=3, entanglement='sca')),
                  quantum_instance=quantum_instance),
    QuantumKernel(feature_map=(ZZFeatureMap(feature_dimension=feature_dimension, reps=4, entanglement='sca')),
                  quantum_instance=quantum_instance),
    QuantumKernel(feature_map=PauliFeatureMap(feature_dimension=feature_dimension, reps=1, entanglement='linear', paulis=['ZZ']),
                  quantum_instance=quantum_instance),
    QuantumKernel(feature_map=PauliFeatureMap(feature_dimension=feature_dimension, reps=1, entanglement='linear', paulis=['Z', 'XX']),
                  quantum_instance=quantum_instance),
    # QuantumKernel(feature_map=PauliFeatureMap(feature_dimension=feature_dimension, reps=2, entanglement='linear', paulis=['ZZ']),
    #               quantum_instance=quantum_instance),
    # QuantumKernel(feature_map=PauliFeatureMap(feature_dimension=feature_dimension, reps=2, entanglement='linear', paulis=['Z', 'XX']),
    #               quantum_instance=quantum_instance),
    # FidelityQuantumKernel(feature_map=ZZFeatureMap(feature_dimension=feature_dimension, reps=2, entanglement='linear'),
    #                       fidelity=ComputeUncompute(sampler=Sampler())),
    # FidelityQuantumKernel(feature_map=ZZFeatureMap(feature_dimension=feature_dimension, reps=3, entanglement='linear'),
    #                       fidelity=ComputeUncompute(sampler=Sampler())),
    # FidelityQuantumKernel(feature_map=PauliFeatureMap(feature_dimension=feature_dimension, reps=1, entanglement='linear', paulis=['Z', 'XX']),
    #                       fidelity=ComputeUncompute(sampler=Sampler())),
    # FidelityQuantumKernel(feature_map=PauliFeatureMap(feature_dimension=feature_dimension, reps=1, entanglement='linear', paulis=['Z', 'XX']),
    #                       fidelity=ComputeUncompute(sampler=Sampler()))
]

In [16]:
# svc = SVC()
# search = RandomizedSearchCV(svc, cv=10, n_iter=16, n_jobs=-1, refit=True,
#                             param_distributions={'kernel': [kernel.evaluate for kernel in kernels]})
# # search.fit(train_features, train_labels)
# search.fit(all_features, all_labels)

In [17]:
# best_kernel = search.best_params_['kernel'].__self__
# print(f"Best is kernel {kernels.index(best_kernel)} using {best_kernel.feature_map.__class__.__name__} with {best_kernel.feature_map.reps} reps and {best_kernel.feature_map.entanglement} entanglement")
# svc = search.best_estimator_
# best_kernel.feature_map.decompose().draw(output='mpl')

## Train the winning kernel circuit using 10-fold cross validation

In [18]:
best_kernel = kernels[0] # Index from output above
svc = SVC(kernel=best_kernel.evaluate)

results = cross_validate(svc, train_features, train_labels, cv=10, n_jobs=-1, return_estimator=True, return_train_score=True)

In [19]:
resulting_models = results['estimator']
total_times = results['fit_time'] + results['score_time']
print(f"Time consumed for each SVC: {total_times}, mean: {total_times.mean()}")

Time consumed for each SVC: [6412.20864749 6420.8831718  6387.18792653 6258.14580607 6378.63742328
 6330.63391972 6553.25407314 6374.828578   6241.02483201 6481.97609258], mean: 6383.878047060966


In [20]:
# Calculate accuracy on testing dataset

accuracies = np.array([model.score(test_features, test_labels) for model in resulting_models])

### Training evaluation

In [21]:
print(', '.join(map(lambda accuracy: f'{accuracy:.2f}', accuracies)))
print('mean: {:.2f}, std: {:.2f}, mean training time: {:.0f}s'.format(accuracies.mean(), accuracies.std(),
                                                                          np.array(total_times).mean()))

0.73, 0.73, 0.73, 0.73, 0.73, 0.73, 0.73, 0.73, 0.73, 0.73
mean: 0.73, std: 0.00, mean training time: 6384s


---
### Classical Implementation

In [22]:
classical_svc = SVC(kernel='rbf')
classical_results = cross_validate(classical_svc, train_features, train_labels, cv=10, n_jobs=-1, return_estimator=True, return_train_score=True)
classical_results

{'fit_time': array([0.01110315, 0.01115155, 0.01211619, 0.01485062, 0.0108912 ,
        0.01126862, 0.01223302, 0.01055837, 0.01467586, 0.01070046]),
 'score_time': array([0.00255895, 0.00238872, 0.0025599 , 0.00242877, 0.00241232,
        0.00226855, 0.00240469, 0.00243115, 0.00356722, 0.00242853]),
 'estimator': [SVC(),
  SVC(),
  SVC(),
  SVC(),
  SVC(),
  SVC(),
  SVC(),
  SVC(),
  SVC(),
  SVC()],
 'test_score': array([0.73333333, 0.72      , 0.68      , 0.73333333, 0.72      ,
        0.69333333, 0.70666667, 0.72      , 0.73333333, 0.70666667]),
 'train_score': array([0.71111111, 0.71259259, 0.71703704, 0.71111111, 0.71111111,
        0.71555556, 0.71555556, 0.71259259, 0.71111111, 0.71407407])}

In [23]:
print(f"Classical mean SVC score: {classical_results['test_score'].mean()} with std. deviation {classical_results['test_score'].std()}")

Classical mean SVC score: 0.7146666666666667 with std. deviation 0.017074997966487565


In [24]:
classical_scores = np.array([model.score(test_features, test_labels) for model in classical_results['estimator']])
classical_total_times = results['fit_time'] + results['score_time']
print(', '.join(map(lambda accuracy: f'{accuracy:.2f}', classical_scores)))
print('mean: {:.2f}, std: {:.2f}, mean training time: {:.0f}s'.format(classical_scores.mean(), classical_scores.std(),
                                                                          np.array(classical_total_times).mean()))

0.72, 0.72, 0.70, 0.72, 0.70, 0.72, 0.70, 0.71, 0.72, 0.72
mean: 0.71, std: 0.01, mean training time: 6384s


---
### Run on real quantum computer

In [ ]:
from qiskit import IBMQ

# Best kernel should already be evaluated and set (in the above code cell)

IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-education', group='zhaw-1')
backend = least_busy(provider.backends(simulator=True))
print(f"Chosen backend: {backend}")
best_model = resulting_models[np.argmax(accuracies)]
best_model.kernel.__self__.quantum_instance = QuantumInstance(backend, shots=1024)
score = best_model.score(test_features, test_labels)
print(f"Quantum score: {score}")


Chosen backend: ibmq_qasm_simulator
